In [11]:
import cv2

# Path to the image file
image_path = r"C:\Users\MAHESH\Pictures\ATIFR 2022\243.jpeg"

# Load image
image = cv2.imread(image_path)

# Check if the image was successfully loaded
if image is None:
    print("Error: Unable to load image.")
else:
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours on original image
    cv2.drawContours(image, contours, -1, (0, 255, 0), 2)

    # Display results
    cv2.imshow('Detected Obstacles', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [13]:
import cv2

# Function to detect obstacles and provide navigation feedback
def detect_obstacles_and_navigate(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours on original image
    cv2.drawContours(image, contours, -1, (0, 255, 0), 2)

    # Check for obstacles
    if len(contours) > 0:
        # If obstacles are detected, provide feedback to user
        print("Obstacle detected. Please navigate around it.")
        # Add code here to provide auditory feedback to the user (e.g., using speech synthesis)

    else:
        # If no obstacles are detected, provide navigation instructions
        print("No obstacles detected. Proceed straight ahead.")
        # Add code here to provide auditory feedback to the user (e.g., using speech synthesis)

    # Display results
    cv2.imshow('Detected Obstacles', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Load image
image_path = r"C:\Users\MAHESH\Pictures\Camera Roll\2.png"
image = cv2.imread(image_path)

# Check if the image was successfully loaded
if image is None:
    print("Error: Unable to load image.")
else:
    # Detect obstacles and provide navigation feedback
    detect_obstacles_and_navigate(image)


Obstacle detected. Please navigate around it.


In [21]:
# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Function to perform object detection
def detect_objects(frame):
    height, width, channels = frame.shape

    # Convert frame to blob for YOLO input
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    # Pass blob through YOLO network
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Initialize lists for object names and their corresponding confidences
    class_ids = []
    confidences = []
    object_names = []

    # Iterate through each detected object
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            # Filter out low confidence detections
            if confidence > 0.5:
                # Object detected, determine its name
                object_name = classes[class_id]
                object_names.append(object_name)
                confidences.append(float(confidence))

    # Display object names and their confidences
    for i, object_name in enumerate(object_names):
        print(f"Object {i+1}: {object_name} (Confidence: {confidences[i]:.2f})")

    # Display results on the frame
    for i in range(len(object_names)):
        cv2.putText(frame, object_names[i], (50, 50 + 30*i), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    return frame

# Open video capture device
video_path = r"C:\Users\MAHESH\Videos\VID20220428102215.mp4"   # Change this to the path of your video file
cap = cv2.VideoCapture(video_path)

# Check if the video capture device is opened successfully
if not cap.isOpened():
    print("Error: Unable to open video.")
else:
    while True:
        # Read a frame from the video feed
        ret, frame = cap.read()

        # Check if frame is successfully read
        if not ret:
            print("Error: Unable to read frame.")
            break
        
        # Perform object detection on the frame
        detected_frame = detect_objects(frame)

        # Display the detected frame
        cv2.imshow('Object Detection', detected_frame)

        # Check for key press to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release video capture device and close all windows
cap.release()
cv2.destroyAllWindows()

Object 1: person (Confidence: 0.62)
Object 1: person (Confidence: 0.70)
Object 1: motorbike (Confidence: 0.51)


In [23]:
!pip install pyttsx3


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import cv2
import numpy as np
import math
import pyttsx3

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Function to calculate distance between two points
def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Function to perform object detection
def detect_objects(frame):
    height, width, channels = frame.shape
    person_detected = False

    # Convert frame to blob for YOLO input
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    # Pass blob through YOLO network
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Initialize lists for object names, positions, and confidences
    object_names = []
    object_positions = []
    confidences = []

    # Iterate through each detected object
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            # Filter out low confidence detections
            if confidence > 0.5:
                # Object detected, determine its name and position
                object_name = classes[class_id]
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                object_names.append(object_name)
                object_positions.append((center_x, center_y))
                confidences.append(float(confidence))
                if object_name == 'person':
                    person_detected = True

    # Perform distance calculation if a person is detected
    if person_detected:
        for i, object_name in enumerate(object_names):
            if object_name != 'person':
                person_x, person_y = object_positions[0]
                object_x, object_y = object_positions[i]
                distance = calculate_distance(person_x, person_y, object_x, object_y)

                # Display information
                info_text = f"{object_name} is {'right' if object_x > person_x else 'left'} of you, {int(distance)} feet away"
                print(info_text)

                # Convert information to speech
                engine.say(info_text)
                engine.runAndWait()

    # Display results on the frame
    for i in range(len(object_names)):
        cv2.putText(frame, object_names[i], object_positions[i], cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    return frame

# Open video capture device
video_path = r"C:\Users\MAHESH\Busy people walking the city streets in London, HD Stock Footage.mp4" # Change this to the path of your video file
cap = cv2.VideoCapture(video_path)

# Check if the video capture device is opened successfully
if not cap.isOpened():
    print("Error: Unable to open video.")
else:
    while True:
        # Read a frame from the video feed
        ret, frame = cap.read()

        # Check if frame is successfully read
        if not ret:
            print("Error: Unable to read frame.")
            break

        # Perform object detection on the frame
        detected_frame = detect_objects(frame)

        # Display the detected frame
        cv2.imshow('Object Detection', detected_frame)

        # Check for key press to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release video capture device and close all windows
cap.release()
cv2.destroyAllWindows()q

In [1]:
import cv2
import numpy as np
import math
import pyttsx3

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Function to calculate distance between two points
def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Function to determine direction based on object position
def determine_direction(person_x, person_y, object_x, object_y):
    if object_x > person_x + 50:
        return "right"
    elif object_x < person_x - 50:
        return "left"
    elif object_y < person_y:
        return "front"
    elif object_y > person_y:
        return "behind"
    else:
        return "unknown"

# Function to perform object detection
def detect_objects(frame):
    height, width, channels = frame.shape

    # Convert frame to blob for YOLO input
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    # Pass blob through YOLO network
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Initialize lists for object names, positions, and confidences
    object_names = []
    object_positions = []
    confidences = []

    # Detect person's position (assumed to be in the center)
    person_x, person_y = width // 2, height // 2

    # Iterate through each detected object
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            # Filter out low confidence detections
            if confidence > 0.5:
                # Object detected, determine its name and position
                object_name = classes[class_id]
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                object_names.append(object_name)
                object_positions.append((center_x, center_y))
                confidences.append(float(confidence))

    # Keep track of detected objects to avoid duplicates
    detected_objects = set()

    # Iterate through detected objects
    for object_name, object_position in zip(object_names, object_positions):
        # Check if object has already been detected
        if object_name not in detected_objects:
            # Calculate distance and direction
            object_x, object_y = object_position
            distance = calculate_distance(person_x, person_y, object_x, object_y)
            direction = determine_direction(person_x, person_y, object_x, object_y)

            # Display information
            info_text = f"{object_name} is {int(distance)} feet away, {direction} of you"
            print(info_text)

            # Convert information to speech
            engine.say(info_text)
            engine.runAndWait()

            # Add object to detected set
            detected_objects.add(object_name)

    # Exit after processing all detected objects
    return True

# Open video capture device
video_path = r"C:\Users\MAHESH\Busy people walking the city streets in London, HD Stock Footage.mp4"  # Change this to the path of your video file
cap = cv2.VideoCapture(video_path)

# Check if the video capture device is opened successfully
if not cap.isOpened():
    print("Error: Unable to open video.")
else:
    while True:
        # Read a frame from the video feed
        ret, frame = cap.read()

        # Check if frame is successfully read
        if not ret:
            print("Error: Unable to read frame.")
            break

        # Perform object detection on the frame
        if detect_objects(frame):
            # Exit after detecting all objects
            break

# Release video capture device
cap.release()
cv2.destroyAllWindows()

truck is 315 feet away, left of you
person is 650 feet away, right of you
traffic light is 621 feet away, left of you
bus is 487 feet away, right of you
backpack is 350 feet away, right of you
handbag is 596 feet away, right of you


In [3]:
import cv2
import numpy as np
import math
import pyttsx3

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Function to calculate distance between two points
def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Function to determine direction based on object position
def determine_direction(person_x, person_y, object_x, object_y):
    if object_x > person_x + 50:
        return "right"
    elif object_x < person_x - 50:
        return "left"
    elif object_y < person_y:
        return "front"
    elif object_y > person_y:
        return "behind"
    else:
        return "unknown"

# Function to perform object detection
def detect_objects(frame, max_distance):
    height, width, channels = frame.shape

    # Convert frame to blob for YOLO input
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    # Pass blob through YOLO network
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Initialize lists for object names, positions, and confidences
    object_names = []
    object_positions = []
    confidences = []

    # Detect person's position (assumed to be in the center)
    person_x, person_y = width // 2, height // 2

    # Iterate through each detected object
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            # Filter out low confidence detections
            if confidence > 0.5:
                # Object detected, determine its name and position
                object_name = classes[class_id]
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                object_names.append(object_name)
                object_positions.append((center_x, center_y))
                confidences.append(float(confidence))

    # Keep track of detected objects to avoid duplicates
    detected_objects = set()

    # Iterate through detected objects
    for object_name, object_position in zip(object_names, object_positions):
        # Check if object has already been detected
        if object_name not in detected_objects:
            # Calculate distance and direction
            object_x, object_y = object_position
            distance = calculate_distance(person_x, person_y, object_x, object_y)
            direction = determine_direction(person_x, person_y, object_x, object_y)

            # Check if object is within max distance
            if distance <= max_distance:
                # Display information
                info_text = f"{object_name} is {int(distance)} feet away, {direction} of you"
                print(info_text)

                # Convert information to speech
                engine.say(info_text)
                engine.runAndWait()

                # Add object to detected set
                detected_objects.add(object_name)

    # Exit after processing all detected objects
    return True

# Open video capture device
video_path = r"C:\Users\MAHESH\Busy people walking the city streets in London, HD Stock Footage.mp4"  # Change this to the path of your video file
cap = cv2.VideoCapture(video_path)

# Check if the video capture device is opened successfully
if not cap.isOpened():
    print("Error: Unable to open video.")
else:
    while True:
        # Read a frame from the video feed
        ret, frame = cap.read()

        # Check if frame is successfully read
        if not ret:
            print("Error: Unable to read frame.")
            break

        # Perform object detection on the frame with max distance of 10 feet
        if detect_objects(frame, max_distance=10):
            # Display the frame
            cv2.imshow('Object Detection', frame)

            # Check for key press to exit
            if cv2.waitKey(0) & 0xFF == ord('q'):
                break

# Release video capture device
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import math
import pyttsx3

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Function to calculate distance between two points
def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Function to determine direction based on object position
def determine_direction(person_x, person_y, object_x, object_y):
    if object_x > person_x + 50:
        return "right"
    elif object_x < person_x - 50:
        return "left"
    elif object_y < person_y:
        return "front"
    elif object_y > person_y:
        return "behind"
    else:
        return "unknown"

# Function to perform object detection
def detect_objects(frame, max_distance):
    height, width, channels = frame.shape

    # Convert frame to blob for YOLO input
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    # Pass blob through YOLO network
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Initialize lists for object names, positions, and confidences
    object_names = []
    object_positions = []
    confidences = []

    # Detect person's position (assumed to be in the center)
    person_x, person_y = width // 2, height // 2

    # Iterate through each detected object
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            # Filter out low confidence detections
            if confidence > 0.5:
                # Object detected, determine its name and position
                object_name = classes[class_id]
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                object_names.append(object_name)
                object_positions.append((center_x, center_y))
                confidences.append(float(confidence))

    # Keep track of detected objects to avoid duplicates
    detected_objects = set()

    # Iterate through detected objects
    for object_name, object_position in zip(object_names, object_positions):
        # Check if object has already been detected
        if object_name not in detected_objects:
            # Calculate distance and direction
            object_x, object_y = object_position
            distance = calculate_distance(person_x, person_y, object_x, object_y)
            direction = determine_direction(person_x, person_y, object_x, object_y)

            # Check if object is within max distance
            if distance <= max_distance:
                # Display information
                info_text = f"{object_name} is {int(distance)} feet away, {direction} of you"
                print(info_text)

                # Convert information to speech
                engine.say(info_text)
                engine.runAndWait()

                # Add object to detected set
                detected_objects.add(object_name)

    # Exit after processing all detected objects
    return True

# Open video capture device
video_path = r"C:\Users\MAHESH\Busy people walking the city streets in London, HD Stock Footage.mp4"  # Change this to the path of your video file
cap = cv2.VideoCapture(video_path)

# Check if the video capture device is opened successfully
if not cap.isOpened():
    print("Error: Unable to open video.")
else:
    while True:
        # Read a frame from the video feed
        ret, frame = cap.read()

        # Check if frame is successfully read
        if not ret:
            print("Error: Unable to read frame.")
            break

        # Perform object detection on the frame with max distance of 10 feet
        if detect_objects(frame, max_distance=1000):
            # Display the frame
            cv2.imshow('Object Detection', frame)

            # Check for key press to exit
            if cv2.waitKey(0) & 0xFF == ord('q'):
                break

# Release video capture device
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import math
import pyttsx3

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Function to calculate distance between two points
def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Function to determine direction based on object position
def determine_direction(person_x, person_y, object_x, object_y):
    if object_x > person_x + 50:
        return "right"
    elif object_x < person_x - 50:
        return "left"
    elif object_y < person_y:
        return "front"
    elif object_y > person_y:
        return "behind"
    else:
        return "unknown"

# Function to perform object detection
def detect_objects(frame, max_distance):
    height, width, channels = frame.shape

    # Convert frame to blob for YOLO input
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    # Pass blob through YOLO network
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Initialize lists for object names, positions, and confidences
    object_names = []
    object_positions = []
    confidences = []

    # Detect person's position (assumed to be in the center)
    person_x, person_y = width // 2, height // 2

    # Iterate through each detected object
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            # Filter out low confidence detections
            if confidence > 0.5:
                # Object detected, determine its name and position
                object_name = classes[class_id]
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                object_names.append(object_name)
                object_positions.append((center_x, center_y))
                confidences.append(float(confidence))

    # Keep track of detected objects to avoid duplicates
    detected_objects = set()

    # Iterate through detected objects
    for object_name, object_position in zip(object_names, object_positions):
        # Check if object has already been detected
        if object_name not in detected_objects:
            # Calculate distance and direction
            object_x, object_y = object_position
            distance = calculate_distance(person_x, person_y, object_x, object_y)
            direction = determine_direction(person_x, person_y, object_x, object_y)

            # Check if object is within max distance
            if distance <= max_distance:
                # Display information
                info_text = f"{object_name} is {int(distance)} feet away, {direction} of you"
                print(info_text)

                # Convert information to speech
                engine.say(info_text)
                engine.runAndWait()

                # Draw bounding box around the object
                cv2.rectangle(frame, (object_x - 50, object_y - 50), (object_x + 50, object_y + 50), (0, 255, 0), 2)

                # Add object to detected set
                detected_objects.add(object_name)

    return frame

# Open video capture device
video_path = r"C:\Users\MAHESH\Busy people walking the city streets in London, HD Stock Footage.mp4"  # Change this to the path of your video file
cap = cv2.VideoCapture(video_path)

# Check if the video capture device is opened successfully
if not cap.isOpened():
    print("Error: Unable to open video.")
else:
    while cap.isOpened():
        # Read a frame from the video feed
        ret, frame = cap.read()

        # Check if frame is successfully read
        if not ret:
            print("Error: Unable to read frame.")
            break

        # Perform object detection on the frame with max distance of 10 feet
        detected_frame = detect_objects(frame, max_distance=10)

        # Display the frame
        cv2.imshow('Object Detection', detected_frame)

        # Check for key press to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release video capture device
cap.release()
cv2.destroyAllWindows()
